# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [4]:
from Assets.helpers import get_sample_grid, Grid

In [5]:
grid = get_sample_grid()

In [6]:
import numpy as np

# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [32]:
INITIAL_PARTICLE = 20
def PF(grid: Grid, prob, obs):
    particles = np.full((grid.n * grid.m, ), INITIAL_PARTICLE, dtype=np.int64)
    particles_cnt = grid.n * grid.m * INITIAL_PARTICLE
    first_round = True
    for ob in obs:
        # Elapse Time
        if first_round:
            first_round = False
        else:
            tmp_particles = np.zeros((grid.n * grid.m, ), dtype=np.int64)
            for i, cell in enumerate(particles):
                for j in range(cell):
                    tmp_particles[grid.get_random_neighbour(i)]+=1
            particles = tmp_particles
        # Weight
        weight = particles.copy().astype(np.float64)/particles_cnt
        weight = weight * prob[:, ob]
        weight = weight / np.sum(weight)
        # Resample
        tmp_particles = np.zeros((grid.n * grid.m, ), dtype=np.int64)
        for i in np.random.choice(range(grid.n * grid.m), size=particles_cnt, p=weight):
            tmp_particles[i] += 1
        particles = tmp_particles
        
    return np.argmax(particles)

In [33]:
from Assets.helpers import PF_checker
PF_checker(PF)

PF PASSED!


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [34]:
def viterbi(grid: Grid, prob, obs):
    cells = grid.n * grid.m
    dp = np.zeros((len(obs) + 1, cells), dtype=np.float64)
    dp[0, : ] = np.full((cells, ), 1.0 / cells, dtype=np.float64)
    par = np.zeros((len(obs) + 1, cells), dtype=np.int32)
    par[0, : ] = np.full((cells, ), -1, dtype=np.int32)

    # trans[i, j] = P(X_j | X_i)
    trans = np.zeros((cells, cells), dtype=np.float64)
    for i in range(cells):
        for j in range(cells):
            if grid.has_path(i, j):
                trans[i, j] = 1.0 / len(grid.get_neighbors(i))
                
    for i, ob in zip(range(1, len(obs)+1), obs):
        for j in range(grid.n * grid.m):
            tmp = trans[:, j] * dp[i-1, :]
            par[i, j] = np.argmax(tmp)
            dp[i, j] = prob[j, ob] * tmp[par[i, j]]
            
    i = np.argmax(dp[-1, :])
    res = [0 for _ in range(len(obs))]
    idx = len(obs) - 1
    while idx >= 0:
        res[idx] = i
        i = par[idx+1, i]
        idx -= 1
    return res

In [35]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

VITERBI PASSED!
